In [3]:
import torch
from torch import nn as nn

import cv2 as cv
import mediapipe as mp

import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from IPython.display import display, Image
from sklearn.preprocessing import MinMaxScaler, Normalizer

from tqdm import tqdm
import os
import sys

import json
from glob import glob
from collections import OrderedDict


In [4]:
df = pd.read_json("data-real/landmarks/all_hand_landmarks.json").transpose()
df.head()

,x,y,z,label
data-real/images/A/001.jpg,"{'0': 0.46490940451622004, '1': 0.555219709873...","{'0': 0.708309531211853, '1': 0.66132336854934...","{'0': -5.075436320112203e-07, '1': -0.03012177...",A
data-real/images/A/002.jpg,"{'0': 0.48861816525459206, '1': 0.633939564228...","{'0': 0.5617042779922481, '1': 0.5200358629226...","{'0': -9.525291488898802e-07, '1': -0.05132721...",A
data-real/images/A/003.jpg,"{'0': 0.37925067543983404, '1': 0.471243590116...","{'0': 0.7275454401969911, '1': 0.7026125192642...","{'0': -7.949893756631354e-07, '1': -0.04028412...",A
data-real/images/A/004.jpg,"{'0': 0.36332768201828003, '1': 0.460623979568...","{'0': 0.7279697656631471, '1': 0.6918165683746...","{'0': -5.866638161933224e-07, '1': -0.02518229...",A
data-real/images/A/005.jpg,"{'0': 0.44915258884429904, '1': 0.587392032146...","{'0': 0.633230507373809, '1': 0.59939664602279...","{'0': -9.324294296675362e-07, '1': -0.04653644...",A


In [5]:
def preprocess_column(series, scaler=MinMaxScaler()):

    # Expand dict into new DataFrame
    df = DataFrame(series.tolist())

    # Scale coordinates
    df = DataFrame(scaler.fit_transform(df))

    # Normalize coordinates
    normalizer = Normalizer()
    df = DataFrame(normalizer.fit_transform(df))

    # Renaming columns
    df.columns = [f"{series.name}{col}"for col in df.columns]

    return df

In [6]:
preprocessed_df = pd.concat([preprocess_column(df['x']), preprocess_column(df['y']), preprocess_column(df['z']), df['label'].reset_index(drop=True)], axis=1)

preprocessed_df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,z12,z13,z14,z15,z16,z17,z18,z19,z20,label
0,0.174138,0.203343,0.229114,0.245728,0.246139,0.236828,0.235357,0.228395,0.228840,0.213760,...,0.231463,0.248762,0.233060,0.236895,0.240368,0.222757,0.217457,0.222375,0.230399,A
1,0.154908,0.199391,0.247105,0.268197,0.270157,0.253825,0.249148,0.234938,0.229938,0.222525,...,0.230862,0.253146,0.231245,0.239760,0.248545,0.227390,0.216762,0.227295,0.241564,A
2,0.159652,0.191880,0.228607,0.257717,0.252847,0.257458,0.247691,0.231889,0.229773,0.232824,...,0.230438,0.250651,0.238889,0.240883,0.241257,0.225991,0.223449,0.227114,0.233759,A
3,0.164240,0.201327,0.234502,0.259028,0.267355,0.246959,0.244995,0.234087,0.236546,0.217758,...,0.231807,0.248318,0.233374,0.239115,0.242942,0.223314,0.217289,0.224675,0.234197,A
4,0.151370,0.195552,0.242214,0.266695,0.271002,0.249770,0.250443,0.233929,0.226981,0.219453,...,0.232041,0.251699,0.231547,0.239487,0.247135,0.224706,0.215417,0.225201,0.238368,A


In [7]:
with open("processed_data.json", "w") as file:
    file.write(preprocessed_df.to_json(indent=4))